In [1]:
import pandas as pd
import re
import sys
import os
import re

In [2]:
data= pd.read_csv("../data/zapa_total.csv")

In [3]:
df = data

# 데이터 필터링

등록된 리뷰내용이 있는 것만 가져오기

In [4]:

filtered_df = df[df["리뷰 내용"] != "등록된 리뷰내용이 없습니다"]


In [5]:
df = filtered_df

# 데이터 전처리

불용어 제거 & 특수문자 제거

In [6]:
from konlpy.tag import Kkma #형태소 분석기
from konlpy.tag import Okt

kkma = Kkma()
okt = Okt()

In [7]:
from soynlp.word import WordExtractor


# WordExtractor를 사용하여 scores 생성
word_extractor = WordExtractor()
word_extractor.train(df['리뷰 내용'].tolist())
word_scores = word_extractor.extract()

# scores 사전 생성
scores = {word: score.cohesion_forward for word, score in word_scores.items()}

training was done. used memory 0.244 Gby 0.233 Gb
all cohesion probabilities was computed. # words = 1622
all branching entropies was computed # words = 3061
all accessor variety was computed # words = 3061


In [8]:
from soynlp.tokenizer import LTokenizer
# LTokenizer 초기화
ltokenizer = LTokenizer(scores=scores)

### 불용어 리스트

In [9]:
stopwords_lst = ['등록된', '리뷰', '조금', '수', '것', '없습니다', '도움이', '정말', '살짝', '또', '너무', '잘', '같아요', '엄청',  '바로', '한', '계속', '구매', '넣고', '먹기', '있어서', '다', '넘', '저는', '그냥', '맛이', '좋아요', 
                '아주', '펜슬', '좀', '좋고', '진짜', '완전', '있는', '때', '프로타주', '이', '좋아하는', '더', '좋은', '있습니다', '입니다', 
                '같아요', '있어', '좋은', '같습니다', '좋네요', '입니다', '있어요', '괜찮', '아니', '그런',
                '같아', '좋습니다', '좋을', '있는데', '없어', '아니라', '좋은', '같은', '없는', '어요', '좋아', '좋앗', '입니', '있고', '없고',
                '좋았', '좋습니', '생각',  '있을', '있습니', '있었', '아닌', '같습니', '습니', '니다', '정도', '쿠팡']

In [10]:
# sys.path.append(os.path.dirname('PyKoSpacing/'))

# spacing = Spacing()  # Spacing 객체 초기화
# pip install git+https://github.com/haven-jeon/PyKoSpacing.git


def preprocessing(df, review_column, istokenize=True, pos_tags=None, filter=None):
    """
    리뷰 데이터 전처리 함수.
    
    Args:
        df (pd.DataFrame): 입력 데이터프레임.
        review_column (str): 리뷰 내용이 있는 열의 이름.
        pos_tags (list[str]): 추출할 품사 태그 리스트 (예: ['Noun', 'Adjective']).
        istokenize (bool): True일 경우 토큰화 사용, False일 경우 원문을 그대로 처리.
        filter (str): 제거할 단어 리스트 (옵션).

    Returns:
        pd.DataFrame: 전처리된 결과를 추가한 데이터프레임.
    """
    #spaceing 객체 정의 및 초기화
    # spacing = Spacing()
    processed_reviews  = []
    #인풋리뷰
    for idx, r in enumerate(df[review_column]):
        #하나의 리뷰에서 문장 단위로 자르기
        #불필요한 정보제거 name의 각 단어를 review에서 제거 
        # 이름이 주어졌을 경우에만 이름 정보 제거
        sentence = r

        # #spacing 적용 
        # sentence = sentence.replace(" ", '')
        
        # sentence = spacing(sentence) 

        #특수문자, 영어 알파벳, 초성/중성/종성(예: "ㄱ", "ㅏ" 등)을 제거.
        sentence = re.sub('\n','',sentence)
        sentence = re.sub('\u200b','',sentence)
        sentence = re.sub('\xa0','',sentence)
        sentence = re.sub('([a-zA-Z])','',sentence)
        sentence = re.sub('[ㄱ-ㅎㅏ-ㅣ]+','',sentence)
        sentence = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',sentence)
        sentence = re.sub(r'\s+', ' ', sentence)  # 다중 공백 -> 단일 공백
        sentence = re.sub(r'[^\w\s]', '', sentence)  # 특수문자 제거
        sentence = re.sub(r'\d+', '', sentence)  # 숫자 제거
        #전처리 후 문장이 비어있으면 다음 리뷰로 넘어감 
        if len(sentence) == 0:
            continue
        
        # 토큰화 여부 확인
        if istokenize:
            tokens = ltokenizer.tokenize(sentence)  # LTokenizer를 이용한 토큰화
        else:
            tokens = [sentence]  # 토큰화하지 않고 원문 사용
        
        # 불용어 필터링
        if filter:
            tokens = [token for token in tokens if token not in filter]
        # 품사 필터링
        if pos_tags:
            filtered_tokens = []
            for token in tokens:
                pos = okt.pos(token, norm=True, stem=False)  # 품사 태깅
                filtered_tokens += [word for word, tag in pos if tag in pos_tags]

        else:
            filtered_tokens = tokens


        # 길이가 1인 단어 제거
        filtered_tokens = [token for token in filtered_tokens if len(token) > 1]

        # 토큰을 공백으로 연결하고 마침표 추가
        processed_sentence = ' '.join(filtered_tokens)
        processed_reviews.append(processed_sentence)
        
    return pd.Series(processed_reviews)  # 시리즈 형태로 반환

In [11]:
df['preprocessed_review'] = preprocessing(
    df, 
    review_column='리뷰 내용', 
    istokenize=False # 토큰화 미사용
)

C:\Users\zin\AppData\Local\Temp\ipykernel_29208\2540016584.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['preprocessed_review'] = preprocessing(


특정 단어 제거(이벤트, 체험단 등)

In [12]:
# def remove_rows_with_keywords(dataframe, column_name, keywords):
#     """
#     특정 단어가 포함된 문장을 제거하는 함수.

#     Args:
#         dataframe (pd.DataFrame): 데이터프레임
#         column_name (str): 처리할 열 이름
#         keywords (list): 제거할 단어 리스트

#     Returns:
#         pd.DataFrame: 특정 단어가 포함된 행이 제거된 데이터프레임
#     """
#     # 특정 단어가 포함된 행 제거
#     pattern = "|".join(keywords)  # 키워드 리스트를 정규식 패턴으로 변환
#     filtered_df = dataframe[~dataframe[column_name].str.contains(pattern, na=False)]
#     return filtered_df

# # 사용 예시
# keywords = ["이벤트", "체험단"]
# filtered_data_oranda = remove_rows_with_keywords(data_oranda, "리뷰 내용", keywords)

# # 결과 확인
# print(filtered_data_oranda)
# len(filtered_data_oranda)

# 키워드 뽑기

# Model1: summarize_with_sentences / KR-WordRank

In [13]:
from krwordrank.sentence import summarize_with_sentences

def df_sumWordSentence(df, review_col, rate_col=None, stopwords=None, top_k=5, num_keywords=5, num_keysents=2, full_merge=False, rate_merge=False):
    """
    데이터프레임에서 특정 열을 전처리하고 키워드 추출 결과를 데이터프레임에 저장.

    Parameters:
    - df: pd.DataFrame - 입력 데이터프레임
    - review_col: str - 리뷰 내용을 포함하는 열 이름
    - rate_col: str - 평점 열의 이름 (rate_merge=True인 경우 필요)
    - stopwords: list[str] 불용어어
    - top_k: int - 각 리뷰별 상위 키워드 개수
    - num_keywords: int - 추출할 키워드 개수
    - num_keysents: int - 추출할 중요 문장 개수
    - full_merge: bool - True일 경우 모든 리뷰를 합쳐서 처리
    - rate_merge: bool - True일 경우 평점별로 리뷰를 합쳐서 처리

    Returns:
    - result_df: pd.DataFrame - 키워드 열이 추가된 데이터프레임
    """

    if num_keywords < top_k:
        num_keywords = top_k
        
    def extract_keywords(texts, stopwords=None):
        try:
            keywords, _ = summarize_with_sentences(
                texts,
                stopwords=stopwords,
                num_keywords=num_keywords,
                num_keysents=num_keysents
            )
            top_keywords = sorted(keywords.items(), key=lambda x: x[1], reverse=True)[:top_k]
            return [word for word, _ in top_keywords]
        except ValueError:
            return None

    if full_merge:
        # 모든 리뷰를 하나로 합쳐 처리
        merged_reviews = '. '.join(df[review_col].dropna())
        texts = merged_reviews.split('. ')
        keywords = extract_keywords(texts, stopwords=stopwords)
        result_df = pd.DataFrame({'merge_type': ['full'], 'keywords': [keywords]})
        return result_df

    if rate_merge:
        # 평점별로 리뷰를 합쳐 처리
        if rate_col is None:
            raise ValueError("rate_merge=True 인 경우, rate_col을 지정해야 합니다.")
        
        rate_keywords = []
        for rate, group in df.groupby(rate_col):
            merged_reviews = '. '.join(group[review_col].dropna())
            texts = merged_reviews.split('. ')
            keywords = extract_keywords(texts, stopwords=stopwords)
            rate_keywords.append({'rate': rate, 'keywords': keywords})
        
        result_df = pd.DataFrame(rate_keywords)
        return result_df

    # 기본: 각 리뷰별 처리
    else:
        krwordrank_results = []
        for idx, row in df.iterrows():
            review_content = row[review_col]
            if pd.isna(review_content):
                krwordrank_results.append(None)
                continue
            
            # 문장 리스트
            texts = review_content.split('. ')
            keywords = extract_keywords(texts, stopwords=stopwords)
            krwordrank_results.append(keywords)

        # 새로운 열 추가
        df['krwordrank'] = krwordrank_results
        return df


In [14]:
# 함수 호출
full_merge_df  = df_sumWordSentence(df, review_col='preprocessed_review', stopwords=stopwords_lst, top_k=30, full_merge=True)

In [15]:
full_merge_df

,merge_type,keywords
0,full,"[마라, 짜파게티, 매운, 얼얼한, 맛있, 맛을, 기존, 라면, 많이, 분들, 사천..."


# Model2: KR-WordRank / KR-WordRank

krwordrank beta 옵션 변경 필요(하이퍼파라미터 설정값 정할것)  
키워드 개수 조정도 필요

In [16]:
from krwordrank.word import KRWordRank

def df_krwordRank(df, review_col, rate_col=None, stopwords=None, top_k=5, num_keywords=10, min_count=1, max_length=50, full_merge=False, rate_merge=False):
    """
    데이터프레임에서 특정 열을 전처리하고 키워드 추출 결과를 데이터프레임에 저장.

    Parameters:
    - df: pd.DataFrame - 입력 데이터프레임
    - review_col: str - 리뷰 내용을 포함하는 열 이름
    - rate_col: str - 평점 열의 이름 (rate_merge=True인 경우 필요)
    - stopwords: list[str] 불용어 리스트
    - top_k: int - 각 리뷰별 상위 키워드 개수
    - num_keywords: int - 추출할 키워드 개수
    - min_count: int - 단어 최소 등장 빈도
    - max_length: int - 단어 최대 길이
    - full_merge: bool - True일 경우 모든 리뷰를 합쳐서 처리
    - rate_merge: bool - True일 경우 평점별로 리뷰를 합쳐서 처리

    Returns:
    - result_df: pd.DataFrame - 키워드 열이 추가된 데이터프레임
    """
    if num_keywords < top_k:
        num_keywords = top_k

    def extract_keywords(texts, row=None):
        if not texts or len(' '.join(texts).strip()) == 0:  # 텍스트가 비어있거나 공백만 포함된 경우
            print("입력 텍스트가 유효하지 않습니다.")
            return None
        
        # KRWordRank 모델 초기화
        krwordrank = KRWordRank(
            min_count=min_count,
            max_length=max_length,
            verbose=True
        )
        
        try:
            result = krwordrank.extract(
                texts, 
                beta=0.85, 
                max_iter=50, 
                num_keywords=num_keywords
            )
            keywords = result[0]

            if keywords is None:
                print("키워드 추출 실패: 결과가 None입니다.")
                return None
            
            top_keywords = sorted(keywords.items(), key=lambda x: x[1], reverse=True)[:top_k]
            return [word for word, _ in top_keywords]
        
        except Exception as e:
            print(str(row) + " error occurs:")
            print(e)
            return None

    if full_merge:
        # 모든 리뷰를 하나로 합쳐 처리
        merged_reviews = '. '.join(df[review_col].dropna())
        
        # 불용어 제거
        if stopwords:
            for word in stopwords:
                merged_reviews = merged_reviews.replace(word, '')
        
        texts = merged_reviews.split('. ')
        keywords = extract_keywords(texts)
        result_df = pd.DataFrame({'merge_type': ['full'], 'keywords': [keywords]})
        return result_df

    if rate_merge:
        # 평점별로 리뷰를 합쳐 처리
        if rate_col is None:
            raise ValueError("rate_merge=True 인 경우, rate_col을 지정해야 합니다.")
        
        rate_keywords = []
        for rate, group in df.groupby(rate_col):
            merged_reviews = '. '.join(group[review_col].dropna())
            
            # 불용어 제거
            if stopwords:
                for word in stopwords:
                    merged_reviews = merged_reviews.replace(word, '')
            
            texts = merged_reviews.split('. ')
            keywords = extract_keywords(texts)
            rate_keywords.append({'rate': rate, 'keywords': keywords})
        
        result_df = pd.DataFrame(rate_keywords)
        return result_df
    
    else:
        # 기본: 각 리뷰별 처리
        krwordrank_results = []
        for idx, row in df.iterrows():
            review_content = row[review_col]

            if pd.isna(review_content):
                krwordrank_results.append(None)
                continue
            
            # 불용어 제거
            if stopwords:
                for word in stopwords:
                    review_content = review_content.replace(word, '')
            
            # 문장 리스트
            texts = review_content.split('. ')
            keywords = extract_keywords(texts, row=idx)
            krwordrank_results.append(keywords)

        # 새로운 열 추가
        df['krwordrank'] = krwordrank_results
        return df


In [24]:
full_merge_df = df_krwordRank(df, review_col='preprocessed_review', stopwords=stopwords_lst, top_k=50, full_merge=True)

scan vocabs ... 
num vocabs = 19837
done = 12 Early stopped.


In [25]:
full_merge_df

,merge_type,keywords
0,full,"[마라, 짜파게티, 매운, 얼얼, 느낌, 해서, 맛을, 라면, 먹어, 기존, 들어,..."


In [26]:
full_merge_df['keywords'].to_csv('col2_output.txt', index=False)

# 품사 분리

In [31]:
print(len('합'))

1


In [27]:
okt = Okt()

def filter_keywords_by_pos(df, column, pos_tags):
    """
    데이터프레임의 지정 열에서 리스트로 저장된 키워드들에 대해 특정 품사만 남기고, 단어 길이가 1 이하인 단어 제거.

    Args:
        df (pd.DataFrame): 입력 데이터프레임
        column (str): 키워드가 저장된 열 이름
        pos_tags (list[str]): 남길 품사 태그 리스트 (예: ['Noun', 'Adjective'])

    Returns:
        pd.DataFrame: 품사 필터링이 적용된 결과
    """
    filtered_results = []
    df = df.copy()

    for keyword_list in df[column]:
        if not isinstance(keyword_list, list):
            filtered_results.append([])
            continue

        filtered_keywords = []
        for keyword in keyword_list:
            # 품사 태깅
            pos = okt.pos(keyword)
            # 지정한 품사만 필터링
            filtered_keywords += [word for word, tag in pos if tag in pos_tags]

        # 단어 길이가 1인 단어 제거
        filtered_keywords = [word for word in filtered_keywords if len(word) > 1]

        # 결과 저장
        filtered_results.append(filtered_keywords)

    # 데이터프레임에 새로운 열 추가
    df[f'{column}_filtered'] = filtered_results
    return df

In [28]:
# 명사와 대명사사 남기기
pos_tags = ['Noun', 'Pronoun']
df_noun = filter_keywords_by_pos(full_merge_df, column='keywords', pos_tags=pos_tags)

In [29]:
df_noun

,merge_type,keywords,keywords_filtered
0,full,"[마라, 짜파게티, 매운, 얼얼, 느낌, 해서, 맛을, 라면, 먹어, 기존, 들어,...","[짜파게티, 얼얼, 느낌, 맛, 라면, 기존, 짜장, 사천, 일반, 맵, 특유, 쿠..."


In [30]:
# 형용사 남기기
pos_tags = ['Adjective']
df_adj = filter_keywords_by_pos(full_merge_df, column='keywords', pos_tags=pos_tags)
df_adj

,merge_type,keywords,keywords_filtered
0,full,"[마라, 짜파게티, 매운, 얼얼, 느낌, 해서, 맛을, 라면, 먹어, 기존, 들어,...","[마라, 매운, 맛있게, 매워, 새로운]"
